In [5]:
import numpy as np
import pandas as pd


In [6]:
# Using Mashup embedding data With 122 classes
datf1_mashup = np.load("processed-211-labels/processed/mashup.human-features.npy")
label1_mashup = np.load("processed-211-labels/processed/mashup.human-labels.npy")

In [18]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, f1_score,recall_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
import datetime
import pickle

#Takes a scikitlearn classifier (Param: method ) as input along with with features(param: h_features) and lables (Param:h_labels)
# runs The classifier N times (param: val_iter)

def classify(method,h_features, h_labels, filename, val_iter=3):
    
    print(h_features.shape, h_labels.shape)
    print "Training -> {0}  - Classifier running {1} times.................".format(str(method), val_iter)   
    
    f1_macro=[]
    f1_micro=[]
    recall_macro=[]
    recall_micro=[]
    accuracy=[]
    classwise_stats=[]
    precision_macro= []
    precision_micro=[]
    
    for i in range(val_iter):
        print "run - - - - - - - - -  {0} at: {1} ".format(i+1, datetime.datetime.now())
    
        x_Train, x_Test, y_Train,  y_Test = train_test_split(h_features,h_labels,test_size=0.20 ) #Train test split
        classifier = MultiOutputClassifier(method)
        classifier.fit(x_Train,y_Train)
        predicted = classifier.predict(x_Test)
        f1_macro.append(f1_score(y_Test, predicted, average='macro'))
        f1_micro.append(f1_score(y_Test, predicted, average='micro'))
        recall_macro.append(recall_score(y_Test, predicted, average='macro'))
        recall_micro.append(recall_score(y_Test, predicted, average='micro'))
        accuracy.append(accuracy_score(y_Test, predicted))
        precision_macro.append(precision_score(y_Test, predicted, average='macro'))
        precision_micro.append(precision_score(y_Test, predicted, average='micro'))
                         
        classwise_stats.append(class_metrics(predicted,y_Test))
    
    results = {
        'classwise_stats': classwise_stats,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        'recall_macro': recall_macro,
        'recall_micro': recall_micro,
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'precision_micro': precision_micro
    }
    
    with open(filename, 'w') as f:
        pickle.dump(results, f)
    
    print "F1 macro"
    print f1_macro
    print "F1 Micro"
    print f1_micro
    
    print "Recall macro"
    print recall_macro
    print "Recall Micro"
    print recall_micro
    
    print "Precision Macro"
    print precision_macro
    
    print "Precision Micro"
    print precision_micro
           
    print "Accuracy"
    print accuracy
    
    print "Classwise Metrics"
    print classwise_stats
    
    
     
    
    return accuracy, predicted

In [8]:
#Calculates class wise metrics. Input - labels predicted by a classifier (param: predicted) 
# and the true labels ( pram: true_labels)
def class_metrics(predicted, true_labels):
    if predicted.shape != true_labels.shape:
        print "Incompatible shapes. Unable to compute classwise metrics"
    #print "Total Classes: {0}".format(predicted.shape[1])
    f1_macro=[]
    recall_macro=[]
    accuracy=[]
    for k in range(predicted.shape[1]):
        
        recall_macro.append(recall_score(predicted[:,k], true_labels[:,k], average='macro'))
        
        f1_macro.append(f1_score(predicted[:,k], true_labels[:,k], average='macro'))
        
        accuracy.append(accuracy_score(predicted[:,k], true_labels[:,k]))
#     print " "    
#     print " *Classwise Recall macro"
#     print recall_macro
    
#     print "............................. "
#     print " *Classwise F1 Macro"
#     print f1_macro
    
#     print "..............................."
#     print " *Classwise Accuracy"
#     print accuracy
    classwise_stats= {"Recall_macro":recall_macro, "F1_Macro":f1_macro,"Classwise_Accuracy": accuracy  }
    return classwise_stats
    
    
    

In [9]:
from sklearn.svm import SVC
scoresvc, predictedsvc = classify(SVC(C=1.0, kernel='rbf'), datf1_mashup, label1_mashup, 'svc_mashup_211.pkl')


((10110L, 800L), (10110L, 211L))
Training -> SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)  - Classifier running 3 times.................
run - - - - - - - - -  1 at: 2018-11-17 11:58:48.560000 


C:\Users\ijet\Anaconda3\envs\ml_py2\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\ijet\Anaconda3\envs\ml_py2\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\ijet\Anaconda3\envs\ml_py2\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\ijet\Anaconda3\envs\ml_py2\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall'

run - - - - - - - - -  2 at: 2018-11-17 12:26:28.900000 
run - - - - - - - - -  3 at: 2018-11-17 12:52:59.373000 
F1 macro
[0.008439637395097505, 0.00855805705508365, 0.008525508080142841]
F1 Micro
[0.5689488910318226, 0.5781792911744267, 0.5691897011969346]
Recall macro
[0.009478672985781991, 0.009478672985781991, 0.009478672985781991]
Recall Micro
[0.44071709900855627, 0.44563470808784145, 0.4366494913462809]
Accuracy
[0.4297725024727992, 0.4431256181998022, 0.4179030662710188]
Classwise Metrics
[{'Recall_macro': [0.49752720079129575, 0.498269040553907, 0.49134520276953514, 0.49851632047477745, 0.4940652818991098, 0.49727992087042533, 0.49653808110781406, 0.49752720079129575, 0.4977744807121662, 0.498269040553907, 0.4987636003956479, 0.4970326409495549, 0.4957962413452028, 0.4948071216617211, 0.498269040553907, 0.49455984174085066, 0.49629080118694363, 0.491839762611276, 0.49653808110781406, 0.49727992087042533, 0.4957962413452028, 0.49851632047477745, 0.4920870425321464, 0.494559841

In [10]:
from sklearn.linear_model import LogisticRegression
score, predicted = classify(LogisticRegression(), datf1_mashup, label1_mashup, 'logistic_mashup_211.pkl')


((10110L, 800L), (10110L, 211L))
Training -> LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)  - Classifier running 3 times.................
run - - - - - - - - -  1 at: 2018-11-17 13:19:32.473000 


C:\Users\ijet\Anaconda3\envs\ml_py2\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


run - - - - - - - - -  2 at: 2018-11-17 13:25:41.673000 
run - - - - - - - - -  3 at: 2018-11-17 13:32:22.092000 
F1 macro
[0.010806639983646765, 0.01121440625500567, 0.011568096242597192]
F1 Micro
[0.5483898091314219, 0.5456874466268147, 0.5450905147242846]
Recall macro
[0.010491341515034926, 0.010688296870650753, 0.010925683488856483]
Recall Micro
[0.42208128397618433, 0.42039473684210527, 0.41879246250480706]
Accuracy
[0.3961424332344214, 0.3768545994065282, 0.3857566765578635]
Classwise Metrics
[{'Recall_macro': [0.4970326409495549, 0.4967853610286845, 0.4910891089108911, 0.4977744807121662, 0.49332344213649854, 0.49727992087042533, 0.49629080118694363, 0.4990108803165183, 0.498269040553907, 0.4977744807121662, 0.498269040553907, 0.4955467590301831, 0.49529935675408215, 0.49455984174085066, 0.4977744807121662, 0.49406234537357746, 0.49431256181998023, 0.491839762611276, 0.4960435212660732, 0.49777337951509154, 0.4928288822947577, 0.4985155863433944, 0.491835724888669, 0.49604352126

In [19]:
from sklearn.naive_bayes import GaussianNB
score, predicted = classify(GaussianNB(),datf1_mashup, label1_mashup, 'gnb_mashup_211.pkl')


((10110L, 800L), (10110L, 211L))
Training -> GaussianNB(priors=None, var_smoothing=1e-09)  - Classifier running 2 times.................
run - - - - - - - - -  1 at: 2018-11-17 15:32:42.087000 
run - - - - - - - - -  2 at: 2018-11-17 15:33:47.956000 


C:\Users\ijet\Anaconda3\envs\ml_py2\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1 macro
[0.028885956884372205, 0.034668726214190156]
F1 Micro
[0.08398573215361096, 0.08744830941376794]
Recall macro
[0.20350491722682967, 0.2180528305542722]
Recall Micro
[0.4746694593533185, 0.4734623995785592]
Precision Macro
[0.02058235203551049, 0.02428808677502171]
Precision Micro
[0.046068429277465094, 0.04817291328875608]
Accuracy
[0.01730959446092977, 0.017804154302670624]
Classwise Metrics
[{'Recall_macro': [0.504282316816184, 0.49693251533742333, 0.500773591315967, 0.5004807662444951, 0.5009854644000985, 0.5064052171015424, 0.49624060150375937, 0.4982630272952854, 0.5027699499638263, 0.49840565848233587, 0.49818820240314565, 0.5014115632714821, 0.5010695501896726, 0.5024549865917507, 0.5002059212510731, 0.499939348132389, 0.49458030864880287, 0.5018197111020639, 0.5007592335031191, 0.5000518650678395, 0.49556889651229274, 0.49967642023260067, 0.5016784385205437, 0.5131175361407919, 0.5041104497354497, 0.4969262295081967, 0.49020226185297955, 0.4989969135802469, 0.502866140

In [12]:
# from sklearn.ensemble import GradientBoostingClassifier
# score, predicted = classify(GradientBoostingClassifier())


### Predictions on Graph Sage embeddings

In [13]:
# Using Graph sage embedding data with 211 classes
dat1_graph_sage = np.load("processed-graphsage-mean-211-labels/graphsage-mean.human-features.npy")
label1_graph_sage = np.load("processed-graphsage-mean-211-labels/graphsage-mean.human-labels.npy")

In [14]:
from sklearn.naive_bayes import GaussianNB
score, predicted = classify(GaussianNB(), dat1_graph_sage,label1_graph_sage, 'gnb_graphsage_211.pkl')

((10110L, 800L), (10110L, 211L))
Training -> GaussianNB(priors=None, var_smoothing=1e-09)  - Classifier running 3 times.................
run - - - - - - - - -  1 at: 2018-11-17 13:41:00.811000 
run - - - - - - - - -  2 at: 2018-11-17 13:41:31.409000 
run - - - - - - - - -  3 at: 2018-11-17 13:42:01.650000 
F1 macro
[0.024701593502948288, 0.023537927499372387, 0.027236338383413744]
F1 Micro
[0.04360275933879995, 0.04071310829686807, 0.053250520424529914]
Recall macro
[0.3276690751918822, 0.3196691390510222, 0.3436134421255014]
Recall Micro
[0.44241944004226097, 0.40545886075949367, 0.5098013591217982]
Accuracy
[0.0, 0.0, 0.0]
Classwise Metrics
[{'Recall_macro': [0.5036404790903317, 0.5042447715258453, 0.49855728371821, 0.4999806230841074, 0.4978716994433833, 0.5002956738250856, 0.4985233014270705, 0.49872935196950446, 0.5013409022842985, 0.49794913863822804, 0.49882995319812795, 0.5012178517397882, 0.49637478065180785, 0.5018714701394034, 0.49868729880418955, 0.4984606662025164, 0.50002

In [15]:
from sklearn.svm import SVC
scoresvc, predictedsvc = classify(SVC(C=1.0, kernel='rbf'), dat1_graph_sage,label1_graph_sage, 'svc_graphsage_211.pkl')

((10110L, 800L), (10110L, 211L))
Training -> SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)  - Classifier running 3 times.................
run - - - - - - - - -  1 at: 2018-11-17 13:42:32.073000 
run - - - - - - - - -  2 at: 2018-11-17 13:51:17.576000 
run - - - - - - - - -  3 at: 2018-11-17 13:59:57.905000 
F1 macro
[0.008555214309036769, 0.008504174657150513, 0.008525508080142841]
F1 Micro
[0.5689360246322271, 0.5574381565570993, 0.5754831969353996]
Recall macro
[0.009478672985781991, 0.009478672985781991, 0.009478672985781991]
Recall Micro
[0.4348849372384937, 0.42396907216494845, 0.444101048105348]
Accuracy
[0.4248269040553907, 0.4208704253214639, 0.4317507418397626]
Classwise Metrics
[{'Recall_macro': [0.4960435212660732, 0.4960435212660732, 0.49159248269040556, 0.49851632047477745, 0.4955489614

In [16]:
from sklearn.linear_model import LogisticRegression
score, predicted = classify(LogisticRegression(), dat1_graph_sage,label1_graph_sage, 'logistic_graphsage_211.pkl')

((10110L, 800L), (10110L, 211L))
Training -> LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)  - Classifier running 3 times.................
run - - - - - - - - -  1 at: 2018-11-17 14:08:54.875000 
run - - - - - - - - -  2 at: 2018-11-17 14:12:14.079000 
run - - - - - - - - -  3 at: 2018-11-17 14:15:43.365000 
F1 macro
[0.008525494173003556, 0.00848273464780946, 0.008550996531550783]
F1 Micro
[0.5682111235278948, 0.5646064994397034, 0.5686660391888423]
Recall macro
[0.009478672985781991, 0.009478672985781991, 0.009478672985781991]
Recall Micro
[0.4354987481881671, 0.43337303162630675, 0.434776920057569]
Accuracy
[0.4277942631058358, 0.43867457962413453, 0.4307616221562809]
Classwise Metrics
[{'Recall_macro': [0.4970326409495549, 0.4967853610286845, 0.4930761622156281, 0.4975